In [2]:
import numpy as np
import cvxpy as cp
import random
import pandas as pd
import math

In [3]:
n = 10000    # number of bidders
m = 10       # type of resources

A = pd.read_csv("A_3.csv")
pi = pd.read_csv("pi_3.csv")
pi = np.array(pi)
pi = pi.T
A = np.array(A)
b = np.array([1000 for _ in range(m)])  



In [4]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)
prob.solve()
opv1 = prob.value
print("offline optimal value, ", opv1)

offline optimal value,  56275.0128878739


In [8]:
truth = np.array([ 1.04581214,  2.05604972,  3.06239824,  4.07860401,  5.09664173,
        6.06381518,  7.06742707,  8.07094188,  9.0689332 , 10.08168534])
truth

array([ 1.04581214,  2.05604972,  3.06239824,  4.07860401,  5.09664173,
        6.06381518,  7.06742707,  8.07094188,  9.0689332 , 10.08168534])

In [9]:
def AHDL():
    l_decision = []
    b_k = b
    price = [math.inf for _ in range(m)]
    x = 0
    flag = 1
    AT = A.T
    for k in range(n):
        flag = 1
        # Check whether remaining certain resources needed for k
        for i in range(len(b_k)):
            if A[:,k][i] > b_k[i]:
                # print("cannot satisfy")
                l_decision.append(0)
                flag = 0
                break
        if flag == 0:
            continue

        # Decide whether to accept (k)th request and the remaining resource
        if pi[0][k] > AT[k].dot(price):
            x = 1
            b_k = b_k - A[:,k]
        else:
            x = 0
        l_decision.append(x)

        # Update the dual price after making the decision
        A1 = A[:, :k+1]
        pik = pi[:,:k+1]
        print("ak", A[:,k+1])
        print("bk", b_k) 
        Anow = A1.T
        y = cp.Variable((m+k+1))
        e = np.ones(k+1)
        bnow = b_k * (float((k+1)/(n-k+1)))
        pNow = np.reshape(pik, (k+1,)) 
        # print(bnow.shape)
        # print(y[:m].shape)
        # print(e.shape)
        # print(y[m:].shape)
        obj = cp.Minimize(bnow@(y[:m]) + e@(y[m:]))
        cons = [(Anow@(y[:m])+ (y[m:]))>= pNow, y >= 0]
        prob = cp.Problem(obj, cons)
        prob.solve()
        price = y[:m].value
        print("k=",k+1,pik@l_decision-((k+1)/n)*opv1)
        print(price)
        dual_price.append(np.linalg.norm(price-truth))
    return l_decision


In [10]:
dual_price = []
l_decision = AHDL()

ak [1. 1. 0. 1. 0. 0. 1. 0. 1. 1.]
bk [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]
k= 1 [-5.62750129]
[1.84659727e-09 5.80463189e+00 1.84659727e-09 5.80463189e+00
 5.80463189e+00 5.80463189e+00 1.84659727e-09 5.80463189e+00
 1.84659727e-09 5.80463189e+00]
ak [1. 1. 1. 0. 1. 1. 0. 1. 1. 0.]
bk [ 999.  999. 1000.  999. 1000. 1000.  999. 1000.  999.  999.]
k= 2 [21.66936835]
[1.60491614e-09 1.16092636e+01 1.37478714e-09 1.16092636e+01
 1.78226698e-07 1.78226698e-07 1.60491614e-09 1.78226698e-07
 1.60491614e-09 1.16092636e+01]
ak [0. 0. 0. 1. 0. 1. 1. 1. 0. 1.]
bk [998. 998. 999. 999. 999. 999. 999. 999. 998. 999.]
k= 3 [49.68008195]
[5.83228641e-10 3.48277899e+01 6.31761059e-10 2.48203730e-07
 3.09745487e-07 3.09745487e-07 5.97579713e-10 3.09745487e-07
 5.83228641e-10 2.48203730e-07]
ak [1. 0. 1. 0. 0. 0. 1. 0. 1. 0.]
bk [998. 998. 999. 998. 999. 998. 998. 998. 998. 998.]
k= 4 [79.44702747]
[4.86102688e+00 5.86201575e+00 1.85742598e-10 6.22463396e+00
 2.28301550e-10 9.02707269e+00 

In [ ]:
dual_price.to_csv('Q6dualprice.csv',index=False)

In [6]:

opv2 = (np.dot(pi, l_decision)[0])
print("optimal value:", opv2)
print("competetive ratio", opv2/opv1)
        

optimal value: 11349776.349778097
competetive ratio 0.26860138305422826


In [7]:
x = cp.Variable(n)
ee = np.ones(n)
objective = cp.Maximize(pi@x)
constraints = [A@x <= b, x >= 0, x <= ee]
prob = cp.Problem(objective, constraints)

prob.solve()
x.value
opv1 = prob.value
print("competetive ratio", opv2/opv1)

competetive ratio 0.26860138305422826
